# 데이터 입력 함수

In [ ]:
# 프론트로부터 입력받는 데이터 (JSON)

import json
from datetime import datetime
from pathlib import Path

# 파일 경로 설정
DATA_DIR = Path.cwd() / "mock_data"
SONG_SESSIONS_FILE = DATA_DIR / "song_sessions.json"
PERSONS_AVAILABILITY_FILE = DATA_DIR / "persons_availability.json"
BASE_SCHEDULE_FILE = DATA_DIR / "base_schedule.json"
SESSION_WEIGHT_FILE = DATA_DIR / "session_weight.json"

# 곡별 세션 정보 불러오기
def load_song_sessions() -> dict:
    with open(SONG_SESSIONS_FILE, "r", encoding="utf-8") as f:
        return json.load(f)

# 사람별 가능한 시간 불러오기 (문자열 → datetime 변환)
def load_persons_availability() -> dict:
    with open(PERSONS_AVAILABILITY_FILE, "r", encoding="utf-8") as f:
        raw = json.load(f)
        return {
            name: [datetime.strptime(t, "%Y-%m-%d %H:%M") for t in times]
            for name, times in raw.items()
        }

# 기준 스케줄 불러오기 (문자열 → datetime 변환)
def load_base_schedule() -> list[datetime]:
    with open(BASE_SCHEDULE_FILE, "r", encoding="utf-8") as f:
        times = json.load(f)
        return [datetime.strptime(t, "%Y-%m-%d %H:%M") for t in times]

# 세션별 가중치 불러오기
def load_session_weight() -> dict:
    with open(SESSION_WEIGHT_FILE, "r", encoding="utf-8") as f:
        return json.load(f)


song_sessions = load_song_sessions()
persons_availability = load_persons_availability()
base_schedule = load_base_schedule()
session_weight = load_session_weight()

# 방 개수를 전역변수로 저장
rooms = 3


# 타임테이블 - 인원수 기준 & 가중치


In [21]:
# sons_sessions(곡별 세션 정보)를 person_role(이름별 역할)로 정리한 딕셔너리 반환
def make_person_role(song_sessions: dict) -> dict:
    
    person_role = {}
    for song, sessions in song_sessions.items():
        for session, name in sessions.items():
            if name is None:
                continue
            if name not in person_role:
                person_role[name] = []
            person_role[name].append((song, session))
    return person_role

person_role = make_person_role(song_sessions)
print(person_role)

{'오지원': [('Violet', '보컬'), ('Violet', '기타2'), ('불', '보컬')], '윤진경': [('Violet', '기타1'), ('Holiday', '기타2'), ('Antifreeze', '기타2')], '신정현': [('Violet', '베이스'), ('ride', '베이스'), ('Antifreeze', '베이스'), ('불', '베이스')], '박수빈': [('Violet', '드럼'), ('ride', '드럼')], '곽민욱': [('ride', '보컬'), ('ride', '기타2'), ('찬란', '기타1'), ('불', '기타1')], '손호유': [('ride', '기타1'), ('Antifreeze', '기타1'), ('찬란', '기타2')], '차희태': [('ride', '건반'), ('불', '건반')], '이상혁': [('Holiday', '보컬')], '이재욱': [('Holiday', '기타1')], '송채우': [('Holiday', '베이스'), ('찬란', '베이스')], '장희찬': [('Holiday', '드럼'), ('찬란', '드럼')], '김강현': [('Antifreeze', '보컬'), ('찬란', '보컬')], '김정태': [('Antifreeze', '건반'), ('찬란', '건반')], '함채경': [('Antifreeze', '드럼'), ('불', '드럼')]}


In [25]:
# class Person 만들기
# 속성: name, role, available_time

class Person:
    def __init__(self, name, role, available_time):
        self.name = name
        self.role = role
        self.available_time = available_time

    def __repr__(self):
        return f"Person(name={self.name}, role={self.role}, available_time={self.available_time})"


def make_people(person_role: dict, persons_availability: dict) -> list:
    people = []
    
    for name, roles in person_role.items():
        available_time = persons_availability.get(name, [])  # 없으면 빈 리스트
        person = Person(name, roles, available_time)
        people.append(person)
        
    return people
  
people = make_people(person_role,persons_availability)
print(people)

[Person(name=오지원, role=[('Violet', '보컬'), ('Violet', '기타2'), ('불', '보컬')], available_time=[datetime.datetime(2025, 5, 13, 17, 0), datetime.datetime(2025, 5, 13, 17, 30), datetime.datetime(2025, 5, 13, 18, 0), datetime.datetime(2025, 5, 13, 18, 30), datetime.datetime(2025, 5, 13, 19, 0), datetime.datetime(2025, 5, 13, 19, 30)]), Person(name=윤진경, role=[('Violet', '기타1'), ('Holiday', '기타2'), ('Antifreeze', '기타2')], available_time=[datetime.datetime(2025, 5, 13, 18, 0), datetime.datetime(2025, 5, 13, 18, 30), datetime.datetime(2025, 5, 13, 19, 0), datetime.datetime(2025, 5, 13, 19, 30), datetime.datetime(2025, 5, 15, 18, 0), datetime.datetime(2025, 5, 15, 18, 30), datetime.datetime(2025, 5, 15, 19, 0), datetime.datetime(2025, 5, 15, 19, 30)]), Person(name=신정현, role=[('Violet', '베이스'), ('ride', '베이스'), ('Antifreeze', '베이스'), ('불', '베이스')], available_time=[datetime.datetime(2025, 5, 13, 18, 0), datetime.datetime(2025, 5, 13, 18, 30), datetime.datetime(2025, 5, 13, 19, 0), datetime.datetime(2

In [27]:
# 공통되는 시간w/가중치 찾기
from collections import defaultdict
def sort_time_list(base_schedule: list, people: list, session_weight: dict) -> list:

    result = []

    for time in base_schedule:
        song_info = defaultdict(lambda: {'인원 수': 0, '가중치 합': 0})

        for person in people:
            if time in person.available_time:
                for song, session in person.role:
                    song_info[song]['인원 수'] += 1
                    song_info[song]['가중치 합'] += session_weight.get(session, 0)

        # song_info가 비어 있지 않으면 리스트에 튜플로 추가
        for song, info in song_info.items():
            result.append((time, song, info['인원 수'], info['가중치 합']))
    
    # result -> (시간, 곡, 인원수, 가중치) 튜플을 저장하는 리스트
    return result

time_list = sort_time_list(base_schedule, people, session_weight)
print(time_list)

[(datetime.datetime(2025, 5, 13, 17, 0), 'Violet', 2, 6), (datetime.datetime(2025, 5, 13, 17, 0), '불', 1, 4), (datetime.datetime(2025, 5, 13, 17, 0), 'Holiday', 1, 8), (datetime.datetime(2025, 5, 13, 18, 30), 'Violet', 4, 16), (datetime.datetime(2025, 5, 13, 18, 30), '불', 2, 6), (datetime.datetime(2025, 5, 13, 18, 30), 'Holiday', 4, 28), (datetime.datetime(2025, 5, 13, 18, 30), 'Antifreeze', 3, 12), (datetime.datetime(2025, 5, 13, 18, 30), 'ride', 2, 10), (datetime.datetime(2025, 5, 13, 18, 30), '찬란', 3, 20), (datetime.datetime(2025, 5, 13, 18, 0), 'Violet', 4, 16), (datetime.datetime(2025, 5, 13, 18, 0), '불', 2, 6), (datetime.datetime(2025, 5, 13, 18, 0), 'Holiday', 3, 26), (datetime.datetime(2025, 5, 13, 18, 0), 'Antifreeze', 3, 12), (datetime.datetime(2025, 5, 13, 18, 0), 'ride', 2, 10), (datetime.datetime(2025, 5, 13, 18, 0), '찬란', 2, 18), (datetime.datetime(2025, 5, 13, 18, 30), 'Violet', 4, 16), (datetime.datetime(2025, 5, 13, 18, 30), '불', 2, 6), (datetime.datetime(2025, 5, 13, 

In [ ]:
# 자동 스케줄링 with 인원 겹침 제거
def assign_schedule(time_list: list, person_role: dict, rooms: int) -> dict:
    schedule = defaultdict(list)  # time -> list of (song, count, weight)
    assigned = set()  # (person, time) 중복 방지용

    for time, song, count, weight in sorted(time_list, key=lambda x: (x[0], -x[3])):
        if len(schedule[time]) >= rooms:
            continue

        conflict = False
        for person, roles in person_role.items():
            if (song, _) in roles:
                if (person, time) in assigned:
                    conflict = True
                    break

        if not conflict:
            schedule[time].append((song, count, weight))
            for person, roles in person_role.items():
                if (song, _) in roles:
                    assigned.add((person, time))

    return dict(schedule)

schedule = assign_schedule(time_list, person_role, rooms)
print(schedule)

{datetime.datetime(2025, 5, 13, 17, 0): [('Holiday', 1, 8), ('Violet', 2, 6), ('불', 1, 4)], datetime.datetime(2025, 5, 13, 18, 0): [('Holiday', 3, 26), ('찬란', 2, 18), ('Violet', 4, 16)], datetime.datetime(2025, 5, 13, 18, 30): [('Holiday', 4, 28), ('Holiday', 4, 28), ('찬란', 3, 20)], datetime.datetime(2025, 5, 13, 19, 0): [('Holiday', 4, 24), ('찬란', 3, 20), ('Violet', 4, 16)], datetime.datetime(2025, 5, 13, 19, 30): [('Violet', 4, 16), ('Antifreeze', 3, 12), ('ride', 2, 10)], datetime.datetime(2025, 5, 13, 20, 0): [('Holiday', 2, 6), ('Antifreeze', 2, 3), ('찬란', 2, 3)], datetime.datetime(2025, 5, 13, 20, 30): [('Violet', 1, 2), ('ride', 1, 2), ('Antifreeze', 1, 2)], datetime.datetime(2025, 5, 15, 17, 0): [('Violet', 1, 16), ('ride', 1, 16)], datetime.datetime(2025, 5, 15, 17, 30): [('Violet', 1, 16), ('ride', 1, 16)], datetime.datetime(2025, 5, 15, 18, 0): [('Violet', 2, 24), ('불', 2, 24), ('Antifreeze', 3, 22)], datetime.datetime(2025, 5, 15, 18, 30): [('Violet', 2, 24), ('불', 2, 24), 

In [ ]:
# schedule에서 시간별 곡들을 가중치 내림차순으로 정렬
def sort_schedule(schedule: dict) -> dict:
    sorted_schedule = {}

    for time, song_list in schedule.items():
        sorted_list = sorted(song_list, key=lambda x: x[2], reverse=True)
        sorted_schedule[time] = sorted_list

    return sorted_schedule

sorted_schedule = sort_schedule(schedule)
print(sorted_schedule)

{datetime.datetime(2025, 5, 13, 17, 0): [('Holiday', 1, 8), ('Violet', 2, 6), ('불', 1, 4)], datetime.datetime(2025, 5, 13, 18, 0): [('Holiday', 3, 26), ('찬란', 2, 18), ('Violet', 4, 16)], datetime.datetime(2025, 5, 13, 18, 30): [('Holiday', 4, 28), ('Holiday', 4, 28), ('찬란', 3, 20)], datetime.datetime(2025, 5, 13, 19, 0): [('Holiday', 4, 24), ('찬란', 3, 20), ('Violet', 4, 16)], datetime.datetime(2025, 5, 13, 19, 30): [('Violet', 4, 16), ('Antifreeze', 3, 12), ('ride', 2, 10)], datetime.datetime(2025, 5, 13, 20, 0): [('Holiday', 2, 6), ('Antifreeze', 2, 3), ('찬란', 2, 3)], datetime.datetime(2025, 5, 13, 20, 30): [('Violet', 1, 2), ('ride', 1, 2), ('Antifreeze', 1, 2)], datetime.datetime(2025, 5, 15, 17, 0): [('Violet', 1, 16), ('ride', 1, 16)], datetime.datetime(2025, 5, 15, 17, 30): [('Violet', 1, 16), ('ride', 1, 16)], datetime.datetime(2025, 5, 15, 18, 0): [('Violet', 2, 24), ('불', 2, 24), ('Antifreeze', 3, 22)], datetime.datetime(2025, 5, 15, 18, 30): [('Violet', 2, 24), ('불', 2, 24), 

# 시각화 함수 (터미널 & 콘솔용)


In [ ]:
# 시각화 함수
# 디버깅할 때 데이터 형태 확인용
# 변수에는 schedule, sorted_schedule 사용 가능
def visualize_schedule(schedule: dict) -> str:
    result = ""
    for time in sorted(schedule):
        result += f"[시간: {time}]\n"
        for song, count, weight in schedule[time]:
            result += f"  - {song}\t| 인원 수: {count}\t| 가중치: {weight}\n"
        result += "\n"
    return result

print(visualize_schedule(sorted_schedule))

In [ ]:
# schedule에서 시간마다 참여 가능한 사람들을 시각화하는 함수
def visualize_participation(
    schedule: dict,
    person_role: dict,
    persons_availability: dict
) -> str:
    result = ""

    for time in sorted(schedule):
        result += f"[시간: {time}]\n"

        for song, _, _ in schedule[time]:
            participants = []
            absentees = []

            for person, roles in person_role.items():
                for s, session in roles:
                    if s == song:
                        if time in persons_availability.get(person, []):
                            participants.append(f"{person}({session})")
                        else:
                            absentees.append(f"{person}({session})")

            participants_str = ", ".join(participants) if participants else "없음"
            absentees_str = ", ".join(absentees) if absentees else "없음"

            result += f"- {song}\n"
            result += f"  - 참여 인원: {participants_str}\n"
            result += f"  - 불참 인원: {absentees_str}\n\n"

    return result


print(visualize_participation(schedule, person_role, persons_availability))